In [1]:
from pyspark.sql import SparkSession

if 'spark' in locals() or 'spark' in globals():
    spark.stop()

spark = SparkSession\
    .builder\
    .appName("Análise Pokemons")\
    .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 19:43:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/03 19:43:59 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
%%sparksql --output grid --limit 1000
SELECT * FROM pokeapi.tbl_siver_pokemons

UsageError: Cell magic `%%sparksql` not found.


In [2]:
from pyspark.sql.functions import col, count, count_distinct

In [3]:
pokemons = spark.read.table('pokeapi.tbl_silver_pokemons')
pokemons.show(5)

23/03/14 12:21:50 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/03/14 12:21:50 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/03/14 12:21:53 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---+----------+------+------+-----+---------------+----------+------------+--------------------+
| id|      name|weight|height|order|base_experience|is_default|species_name|         species_url|
+---+----------+------+------+-----+---------------+----------+------------+--------------------+
|  1| bulbasaur|    69|     7|    1|             64|      true|   bulbasaur|https://pokeapi.c...|
|  2|   ivysaur|   130|    10|    2|            142|      true|     ivysaur|https://pokeapi.c...|
|  3|  venusaur|  1000|    20|    3|            263|      true|    venusaur|https://pokeapi.c...|
|  4|charmander|    85|     6|    5|             62|      true|  charmander|https://pokeapi.c...|
|  5|charmeleon|   190|    11|    6|            142|      true|  charmeleon|https://pokeapi.c...|
+---+----------+------+------+-----+---------------+----------+------------+--------------------+
only showing top 5 rows



In [4]:
evolutions = spark.read.table('pokeapi.tbl_silver_evolutions')
evolutions.show(5)

+----------+---+---------------+-------+----------+--------+-----+------+---------+--------+----------+-------------+-------------+----------+-----+----------------+-----------+-----+-------------+----------+--------+----------+---------+
|      name| id|evolution_level|is_baby|evolves_to| trigger|level|gender|hold_item|use_item|beauty_req|happiness_req|affection_req|is_raining|stats|turn_upside_down|time_of_day|trade|party_species|party_type|location|known_move|move_type|
+----------+---+---------------+-------+----------+--------+-----+------+---------+--------+----------+-------------+-------------+----------+-----+----------------+-----------+-----+-------------+----------+--------+----------+---------+
| bulbasaur|  1|              1|  false|   ivysaur|level-up|   16|  null|     null|    null|      null|         null|         null|     false| null|           false|           | null|         null|      null|    null|      null|     null|
|   ivysaur|  2|              2|  false|  ve

# Quais são os maiores Pokémon? Exiba os Pokémon e a altura.

In [5]:
pokemons.select(
    col('name').alias('pokemon'),
    col('height').alias('altura')
)\
    .orderBy(col('altura').desc()).show(5)

+----------+------+
|   pokemon|altura|
+----------+------+
| eternatus|   200|
|   wailord|   145|
|   dondozo|   120|
|celesteela|    92|
|   steelix|    92|
+----------+------+
only showing top 5 rows



In [7]:
query = \
"""
SELECT name AS pokemon, height AS altura
FROM pokeapi.tbl_silver_pokemons
ORDER BY height DESC
LIMIT 5
"""

spark.sql(query).show()

+----------+------+
|   pokemon|altura|
+----------+------+
| eternatus|   200|
|   wailord|   145|
|   dondozo|   120|
|   steelix|    92|
|celesteela|    92|
+----------+------+



# Quantos Pokémon possuem mais de um caminho evolutivo, ou seja, mais de uma segunda ou terceira forma? Informe a quantidade, tomando como base o número distinto de Pokémon na sua primeira forma.

In [90]:
resposta = evolutions\
    .groupBy('name')\
    .agg(count_distinct('evolves_to').alias('evolucoes'))\
    .orderBy(col('evolucoes').desc())\
    .where(col('evolucoes') > 1)\
    .count()

print(f"O número de Pokémon com mais de um caminho evolutivo é {resposta}.")

O número de Pokémon com mais de um caminho evolutivo é 19.


In [15]:
query = \
"""
SELECT COUNT(*) AS resposta
FROM (
    SELECT name, COUNT(DISTINCT evolves_to) AS evolucoes
    FROM pokeapi.tbl_silver_evolutions
    GROUP BY name
    HAVING evolucoes > 1
    ORDER BY evolucoes DESC
)
"""

spark.sql(query).show()

+--------+
|resposta|
+--------+
|      19|
+--------+



# Quais são os Pokémon do tipo gelo (ice) que mais fornecem experiência ao serem derrotados? Leve em consideração apenas os Pokémon que estão em sua forma padrão. Exiba os Pokémon e a experiência que é fornecida.

In [21]:
spark.sql('SHOW TABLES IN pokeapi').show(truncate = False)

+---------+----------------------------------------------+-----------+
|namespace|tableName                                     |isTemporary|
+---------+----------------------------------------------+-----------+
|pokeapi  |tbl_bronze_evolutions                         |false      |
|pokeapi  |tbl_bronze_pokemons                           |false      |
|pokeapi  |tbl_silver_evolutions                         |false      |
|pokeapi  |tbl_silver_pokemon_abilities                  |false      |
|pokeapi  |tbl_silver_pokemon_forms                      |false      |
|pokeapi  |tbl_silver_pokemon_game_indices               |false      |
|pokeapi  |tbl_silver_pokemon_held_items                 |false      |
|pokeapi  |tbl_silver_pokemon_held_items_version_details |false      |
|pokeapi  |tbl_silver_pokemon_moves                      |false      |
|pokeapi  |tbl_silver_pokemon_moves_version_group_details|false      |
|pokeapi  |tbl_silver_pokemon_past_types                 |false      |
|pokea

In [22]:
types = spark.read.table('pokeapi.tbl_silver_pokemon_types')
types.show(5)

+---+----------+---------------+--------------------+
| id|types_slot|types_type_name|      types_type_url|
+---+----------+---------------+--------------------+
|754|         1|          grass|https://pokeapi.c...|
|776|         1|           fire|https://pokeapi.c...|
|776|         2|         dragon|https://pokeapi.c...|
|769|         1|          ghost|https://pokeapi.c...|
|769|         2|         ground|https://pokeapi.c...|
+---+----------+---------------+--------------------+
only showing top 5 rows



In [23]:
pokemons\
    .select('id', 'name', 'base_experience')\
    .join(other = types, on = 'id', how = 'left')\
    .where(col('types_type_name') == 'ice')\
    .select('name', 'base_experience')\
    .orderBy(col('base_experience').desc())\
    .show(5)

+---------+---------------+
|     name|base_experience|
+---------+---------------+
|   kyurem|            330|
| articuno|            290|
|glastrier|            290|
|   regice|            290|
|vanilluxe|            268|
+---------+---------------+
only showing top 5 rows



In [24]:
query = \
"""
SELECT p.name, p.base_experience
FROM pokeapi.tbl_silver_pokemons AS p
LEFT JOIN pokeapi.tbl_silver_pokemon_types AS t
ON p.id = t.id
WHERE t.types_type_name = 'ice'
ORDER BY base_experience DESC
LIMIT 5
"""
spark.sql(query).show()

+---------+---------------+
|     name|base_experience|
+---------+---------------+
|   kyurem|            330|
| articuno|            290|
|   regice|            290|
|glastrier|            290|
|vanilluxe|            268|
+---------+---------------+



# Encontre o golpe (move) que é mais aprendido por Pokémon pelo método level-up e em qual versão do jogo isso acontece, levando em consideração apenas os Pokémon que estão em sua forma padrão. Em seguida, entre os Pokémon que aprendem esse golpe (por level-up e na versão do jogo encontrada no passo anterior), diga quais são os Pokémon que possuem o maior valor do atributo (stat) ‘attack’. Exiba os Pokémon, a versão do jogo em que acontece de mais Pokémon aprenderem o golpe e qual é o valor do atributo ‘attack’ deles.

In [25]:
spark.sql('SHOW TABLES IN pokeapi').show(truncate = False)

+---------+----------------------------------------------+-----------+
|namespace|tableName                                     |isTemporary|
+---------+----------------------------------------------+-----------+
|pokeapi  |tbl_bronze_evolutions                         |false      |
|pokeapi  |tbl_bronze_pokemons                           |false      |
|pokeapi  |tbl_silver_evolutions                         |false      |
|pokeapi  |tbl_silver_pokemon_abilities                  |false      |
|pokeapi  |tbl_silver_pokemon_forms                      |false      |
|pokeapi  |tbl_silver_pokemon_game_indices               |false      |
|pokeapi  |tbl_silver_pokemon_held_items                 |false      |
|pokeapi  |tbl_silver_pokemon_held_items_version_details |false      |
|pokeapi  |tbl_silver_pokemon_moves                      |false      |
|pokeapi  |tbl_silver_pokemon_moves_version_group_details|false      |
|pokeapi  |tbl_silver_pokemon_past_types                 |false      |
|pokea

In [27]:
moves = spark.read.table('pokeapi.tbl_silver_pokemon_moves')
moves.show(5)

+---+---------------+--------------------+
| id|moves_move_name|      moves_move_url|
+---+---------------+--------------------+
|151|          pound|https://pokeapi.c...|
|151|     mega-punch|https://pokeapi.c...|
|151|        pay-day|https://pokeapi.c...|
|151|     fire-punch|https://pokeapi.c...|
|151|      ice-punch|https://pokeapi.c...|
+---+---------------+--------------------+
only showing top 5 rows



In [33]:
moves_details = spark.read.table('pokeapi.tbl_silver_pokemon_moves_version_group_details')
moves_details.show(5)

+---+--------------------------------------------+--------------------------------------------------+-------------------------------------------------+----------------------------------------------+---------------------------------------------+
| id|moves_version_group_details_level_learned_at|moves_version_group_details_move_learn_method_name|moves_version_group_details_move_learn_method_url|moves_version_group_details_version_group_name|moves_version_group_details_version_group_url|
+---+--------------------------------------------+--------------------------------------------------+-------------------------------------------------+----------------------------------------------+---------------------------------------------+
|151|                                           1|                                          level-up|                             https://pokeapi.c...|                                      red-blue|                         https://pokeapi.c...|
|151|               

In [69]:
target = moves\
    .join(moves_details, on='id', how='left')\
    .select(
        'id',
        'moves_move_name',
        'moves_version_group_details_level_learned_at',
        'moves_version_group_details_move_learn_method_name',
        'moves_version_group_details_version_group_name'
    )\
    .where(col('moves_version_group_details_move_learn_method_name') == 'level-up')\
    .groupBy('moves_version_group_details_version_group_name','moves_move_name').count()\
    .orderBy(col('count').desc())\
    .limit(1)\
    .collect()
target_move = target[0][1]
target_vers = target[0][0]

In [100]:
moves\
    .join(moves_details, on='id', how='left')\
    .select(
        'id',
        'moves_move_name',
        'moves_version_group_details_level_learned_at',
        'moves_version_group_details_move_learn_method_name',
        'moves_version_group_details_version_group_name'
    )\
    .where(col('moves_version_group_details_move_learn_method_name') == 'level-up')\
    .groupBy('moves_version_group_details_version_group_name','moves_move_name').count()\
    .orderBy(col('count').desc())\
    .limit(1)\
    .show()

+----------------------------------------------+---------------+-----+
|moves_version_group_details_version_group_name|moves_move_name|count|
+----------------------------------------------+---------------+-----+
|                          ultra-sun-ultra-moon|        protect|13073|
+----------------------------------------------+---------------+-----+



In [71]:
target_move
target_vers

'ultra-sun-ultra-moon'

In [64]:
stats = spark.read.table('pokeapi.tbl_silver_pokemon_stats')
stats.show(5)

+---+---------------+------------+---------------+--------------------+
| id|stats_base_stat|stats_effort|stats_stat_name|      stats_stat_url|
+---+---------------+------------+---------------+--------------------+
|668|             86|           0|             hp|https://pokeapi.c...|
|668|             68|           0|         attack|https://pokeapi.c...|
|668|             72|           0|        defense|https://pokeapi.c...|
|668|            109|           2| special-attack|https://pokeapi.c...|
|668|             66|           0|special-defense|https://pokeapi.c...|
+---+---------------+------------+---------------+--------------------+
only showing top 5 rows



In [83]:
stats.where(col('id') == 668).show()

+---+---------------+------------+---------------+--------------------+
| id|stats_base_stat|stats_effort|stats_stat_name|      stats_stat_url|
+---+---------------+------------+---------------+--------------------+
|668|             86|           0|             hp|https://pokeapi.c...|
|668|             68|           0|         attack|https://pokeapi.c...|
|668|             72|           0|        defense|https://pokeapi.c...|
|668|            109|           2| special-attack|https://pokeapi.c...|
|668|             66|           0|special-defense|https://pokeapi.c...|
|668|            106|           0|          speed|https://pokeapi.c...|
+---+---------------+------------+---------------+--------------------+



In [79]:
from pyspark.sql.functions import sum

In [94]:
stats_pivot = stats\
    .groupBy('id')\
    .pivot('stats_stat_name')\
    .agg(
        sum('stats_base_stat').alias('stat'),\
        sum('stats_effort').alias('effort'))\
    .orderBy('id')

stats_pivot.show(5)

+---+-----------+-------------+------------+--------------+-------+---------+-------------------+---------------------+--------------------+----------------------+----------+------------+
| id|attack_stat|attack_effort|defense_stat|defense_effort|hp_stat|hp_effort|special-attack_stat|special-attack_effort|special-defense_stat|special-defense_effort|speed_stat|speed_effort|
+---+-----------+-------------+------------+--------------+-------+---------+-------------------+---------------------+--------------------+----------------------+----------+------------+
|  1|         49|            0|          49|             0|     45|        0|                 65|                    1|                  65|                     0|        45|           0|
|  2|         62|            0|          63|             0|     60|        0|                 80|                    1|                  80|                     1|        60|           0|
|  3|         82|            0|          83|             0| 

In [99]:
pokemons\
    .join(moves, on='id', how= 'left')\
    .join(stats_pivot, on='id', how= 'left')\
    .where(col('moves_move_name') == target_move)\
    .select('name','attack_stat')\
    .orderBy(col('attack_stat').desc())\
    .show(5)

+---------+-----------+
|     name|attack_stat|
+---------+-----------+
|  kartana|        181|
|rampardos|        165|
|regigigas|        160|
|  slaking|        160|
|  groudon|        150|
+---------+-----------+
only showing top 5 rows



In [26]:
spark.stop()

In [67]:
from pyspark.sql.functions import count, count_distinct